In [ ]:
# (0) 필요 모듈 임포트
from datetime import datetime, timedelta
import pandas as pd
import requests
import json
import yaml
import time

# (1) 개인정보 파일 가져오기
with open('config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
ACCESS_TOKEN_EXPIRED = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
URL_BASE = _cfg['URL_BASE']
HTS_ID = _cfg['HTS_ID']
print(APP_KEY, APP_SECRET, ACCESS_TOKEN, HTS_ID)

# (2) 함수 정의
## 1. 접근 토큰 발급
def get_access_token():
    """ OAuth 인증 > 접근토큰발급 """
    headers = {"content-type": "application/json"}
    body = {
        "grant_type": "client_credentials",
        "appkey": APP_KEY,
        "appsecret": APP_SECRET
    }
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"

    time.sleep(0.05)  # 유량제한 예방 (REST: 1초당 20건 제한)
    res = requests.post(URL, headers=headers, data=json.dumps(body))

    if res.status_code == 200:
        try:
            access_token = res.json().get("access_token")
            access_token_expired = res.json().get("access_token_token_expired")  # 수정된 키
            return access_token, access_token_expired
        except KeyError as e:
            print(f"토큰 발급 중 키 에러 발생: {e}")
            print(res.json())
            return None, None
    else:
        print("접근 토큰 발급이 불가능합니다. 응답 코드:", res.status_code)
        print("응답 내용:", res.json())
        return None, None

In [ ]:
ACCESS_TOKEN, ACCESS_TOKEN_EXPIRED = get_access_token()
ACCESS_TOKEN, ACCESS_TOKEN_EXPIRED

In [3]:
# 3. 호출 반복하여 데이터를 데이터프레임에 저장하는 함수
def fetch_multiple_dates(index_code, start_date, num_calls):
    # 데이터를 저장할 리스트
    all_data = []
    
    # 첫 호출 날짜 설정
    current_date = start_date
    
    for _ in range(num_calls):
        # 데이터 호출
        result = get_domestic_index_data(index_code, current_date)
        
        # API 호출 성공 시
        if result and 'output2' in result:
            # output2가 비어있는지 체크
            if not result['output2']:
                print(f"데이터가 없습니다. 날짜: {current_date}")
                break  # 데이터가 없으면 종료
            
            all_data.extend(result['output2'])
            
            # 마지막 데이터의 "stck_bsop_date" 값을 추출하여 이전 날짜로 설정
            last_stck_bsop_date = result['output2'][-1]['stck_bsop_date']
            current_date = (datetime.strptime(last_stck_bsop_date, "%Y%m%d") - timedelta(days=1)).strftime("%Y%m%d")
        else:
            print(f"API 호출 실패 또는 예상 외의 응답: {result}")
            break  # 호출 실패 시 종료
    
    # 결과를 데이터프레임으로 변환
    df = pd.DataFrame(all_data)
    return df

# 엑셀 파일로 내보내는 함수
def save_to_excel(df, index_code, start_date, end_date):
    # 파일 이름 설정 (index_code, 시작 날짜, 끝 날짜)
    file_name = f"{index_code}_daily_data_{start_date}_{end_date}.xlsx"
    
    # 데이터프레임을 엑셀 파일로 저장
    df.to_excel(file_name, index=False)
    print(f"파일 저장 완료: {file_name}")

In [4]:
# 실행부분
index_code = '0001'  # 예시 인덱스 코드
start_date = datetime.today().strftime('%Y%m%d')  # 시작 날짜 = 오늘날짜
num_calls = 200  # 호출할 횟수

# 데이터 수집
df = fetch_multiple_dates(index_code, start_date, num_calls)

# 첫 번째 및 마지막 데이터 일시 가져오기
if not df.empty:
    first_date = df.iloc[0]['stck_bsop_date']
    last_date = df.iloc[-1]['stck_bsop_date']
    save_to_excel(df, index_code, first_date, last_date)
else:
    print("데이터가 없습니다.")

NameError: name 'get_domestic_index_data' is not defined

In [5]:
df  # 19830104 ~ 20241108 기간의 코스피지수 일별시세

NameError: name 'df' is not defined